# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import modin.pandas as pd 
import cassandra 
import re 
import os 
import glob 
import numpy as np 
import json 
import csv 

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):  
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra
['/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-01-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-02-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-03-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-04-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-05-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-06-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-11-07-events.csv', '/mnt/c/Users/solor/OneDrive/Desktop/ASG/aws/udacity/Data_Modeling_with_Cassandra/event_data/2018-1

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connected to cassandra!")
except Exception as e:
    print(e)

Connected to cassandra!


#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
    print("Keyspace created!")

except Exception as e:
    print(e)


Keyspace created!


#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
    print("keyspace set!")
except Exception as e:
    print(e)

keyspace set!


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
import modin.pandas as pd

df = pd.read_csv("event_datafile_new.csv")
df


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8
...,...,...,...,...,...,...,...,...,...,...,...
6815,Foo Fighters,Rylan,M,57,George,271.38567,paid,"Birmingham-Hoover, AL",1076,The Pretender,16
6816,Timbiriche,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",1076,Besos De Ceniza,16
6817,A Perfect Circle,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",1076,Rose,16
6818,Anberlin,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",1076,The Haunting,16


In [11]:
query1 =  """CREATE TABLE IF NOT EXISTS music_app_history (
    sessionId int,
    itemInSession int,
    artist text,
    song_length float,
    song_title text,
    PRIMARY KEY (sessionId, itemInSession)
);"""
try:
    session.execute(query1)
    print("Table created!")
except Exception as e:
    print(e)
    
query2 =  """CREATE TABLE IF NOT EXISTS user_sessions (
    userId int,
    sessionId int,
    itemInSession int,
    artist text,
    song_title text,
    first_name text,
    last_name text,
    PRIMARY KEY ((userId, sessionId), itemInSession)
) WITH CLUSTERING ORDER BY (itemInSession ASC);"""
try:
    session.execute(query2)
    print("Table created!")
except Exception as e:
    print(e)
    
query3 =  """CREATE TABLE IF NOT EXISTS song_listeners (
    song_title text,
    userId int,
    first_name text,
    last_name text,
    PRIMARY KEY (song_title, userId)
);"""
try:
    session.execute(query3)
    print("Table created!")
except Exception as e:
    print(e)
    


                    

Table created!
Table created!
Table created!


#### Justification
For table music_app_history, the session_id is used as a partition key and item_in_session 
is used as clustering column to help make up a unique key and the queries will filter by these columns.

For table user_sessions, the user_id and session_id are used as composite partition key because the queries will filter by these columns and item_in_session is used as clustering column to help make up a unique key.

For table song_listeners, the song_title is used as a partition key and user_id is used as clustering column to help make up a unique key and the queries will filter by these columns.

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        # Table 1: music_app_history (sessionId, itemInSession, artist, song_title, length)
        query = """
        INSERT INTO music_app_history (sessionId, itemInSession, artist, song_length, song_title)
        VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        

        # Table 2: user_sessions (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)
        query = """
        INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song_title, first_name, last_name)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

        # Table 3: song_listeners (song_title, userId, first_name, last_name)
        query = """
        INSERT INTO song_listeners (song_title, userId, first_name, last_name)
        VALUES (%s, %s, %s, %s)
        """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
    print("Data inserted!")




Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inserted!
Data inser

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:

query1 = """SELECT artist,song_title, song_length FROM music_app_history 
        WHERE sessionid = 338 AND iteminsession = 4;"""
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [14]:
##Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """
SELECT artist, song_title,first_name, last_name 
FROM user_sessions 
WHERE userid = 10 AND sessionid = 182
ORDER BY iteminsession;"""
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.first_name, row.last_name)

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = """
SELECT first_name, last_name 
FROM song_listeners 
WHERE song_title = 'All Hands Against His Own';"""
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
## TO-DO: Drop the table before closing out the sessions
##DROPPING TABLE##
for t in ["music_app_history", "song_listeners", "user_sessions"]:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
        print("Tables Dropped!")
    except Exception as e:
        print(e)

Tables Dropped!
Tables Dropped!
Tables Dropped!


### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()